## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из скачанной открытой гугл-папки

In [1]:
import os
import pandas as pd
path = r'C:\Users\Arslan\Documents\ФКН ПМИ\1 курс\Практика\test\kaggle_data\kaggle_data'
data = {}

Data = []
Target = []

i=0
for dir_entry in os.listdir(path):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            df = pd.read_csv(my_file, delimiter=';')
            if df.shape[1] == 5:
                Data.append(df)
                Target.append(dir_entry_path.split("\\")[-1])

for dir_entry in os.listdir(path):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            df = pd.read_csv(my_file, delimiter=',')
            if df.shape[1] == 5:
                Data.append(df)
                Target.append(dir_entry_path.split("\\")[-1])

In [2]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions

def rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

def integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

Переведём типы движений в числа

In [3]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [4]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [5]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

Action = np.zeros(len(Data))
Ampl = np.zeros(len(Data))
FFT = np.zeros(len(Data))
Dist = np.zeros(len(Data))
FFT_z = np.zeros(len(Data))
good_len = 0


def process_data(index, df, comma=True, cut=True, name = "", a=-1, b=-1):
    
    global good_len, Action, Ampl, FFT, Dist, FFT_z
    good_data = True
    
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
            df['time'] = df['time'].astype(float)
        except KeyError:
            return -1
    
    Fs = round(df.shape[0] / (df.time[df.shape[0] - 1] - df.time[0]))
    
    if a == -1:
        st = 5 * Fs
    else:
        st = a
            
    if b == -1:
        end = -5 * Fs
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

#вычисляем средний вектор ускорения по треку
    try :
        mx = df.iloc[st:end]['gFx'].values.mean()
        my = df.iloc[st:end]['gFy'].values.mean()
        mz = df.iloc[st:end]['gFz'].values.mean()
    except :
        good_data = False
#находим матрицу поворота этого вектора к вектору (0,0,1)

    Vec = [mx,my,mz]
    Point = [0, 0, 1]
    mat = rotation_matrix_from_vectors(Vec, Point)
    
#каждую точку трека поворачиваем на эту матрицу

    new_x, new_y, new_z = rotate(df.iloc[st:end]['gFx'].values, df.iloc[st:end]['gFy'].values, df.iloc[st:end]['gFz'].values, mat)

#проверяем, что новые средние равны примерно (0,0,1). Если это не так, значит, телефон часто поворачивался при движении, и нам не поможет это преобразование

    eps = 0.3
    if len(new_x) == 0 :
        good_data = False
    if len(new_y) == 0 :
        good_data = False
    if len(new_z) == 0 :
        good_data = False
    if good_data and ((abs(mean(new_x)) > eps) or (abs(mean(new_y)) > eps) or (abs(mean(new_z) - 1) > eps)) :
        good_data = False

#делим на 250*250 (это нужно было еще при интегрировании)
    new_x = [elem / (Fs * Fs) for elem in new_x]
    new_y = [elem / (Fs * Fs) for elem in new_y]
#из ускорения по z вычитаем 1 (g), чтобы убрать влияние силы тяжести и оставить только ускорение по z
    new_z = [(elem - 1) / (Fs * Fs) for elem in new_z]
    
#дважды интегрируем ускорение, чтобы получить перемещение вдоль каждой оси
    int_x = integrate(integrate(new_x))
    
    int_y = integrate(integrate(new_y))
    
    int_z = integrate(integrate(new_z))

########################################################################        
#преобразование Фурье
########################################################################        

    yabs_z = []
    try:
        y = df.iloc[st:end]['gFz'].values
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n/Fs
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range

        Y = np.fft.fft(y)/n # dft and normalization
        Y = Y[:n//2]

        yabs_z = abs(Y)
        
    except:
        good_data = False
    
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
    try :
        y = Acc.values
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n/Fs
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range

        Y = np.fft.fft(y)/n # dft and normalization
        Y = Y[:n//2]
    
        yabs = abs(Y)
    except :
        good_data = False
########################################################################        
    
    if good_data :
        good_len += 1
        Action[index] = get_action(name)
        Ampl[index] = Acc.max() - Acc.min()
        FFT[index] = max(yabs[1:2 * Fs])
        FFT_z[index] = max(yabs_z[1:2 * Fs])
        Dist[index] = max(int_z) - min(int_z)
    else :
        print(name)
        Action[index] = get_action(name)
        Ampl[index] = Acc.max() - Acc.min()
        FFT[index] = 1
        Dist[index] = 15

for df, target in zip(Data,Target):
    track_num = int(target.split('_')[1].split('.')[0]) # важный момент, что треки не по нумерации идут!
    process_data(track_num, df, name = target)

Populating the interactive namespace from numpy and matplotlib


C:\Users\Arslan\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


track_11.csv
track_119.csv


<ipython-input-5-c388907ee8ab>:46: RuntimeWarning: Mean of empty slice.
  mx = df.iloc[st:end]['gFx'].values.mean()
C:\Users\Arslan\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-5-c388907ee8ab>:47: RuntimeWarning: Mean of empty slice.
  my = df.iloc[st:end]['gFy'].values.mean()
<ipython-input-5-c388907ee8ab>:48: RuntimeWarning: Mean of empty slice.
  mz = df.iloc[st:end]['gFz'].values.mean()


track_133.csv
track_138.csv
track_140.csv
track_23.csv
track_44.csv
track_45.csv
track_50.csv
track_6.csv
track_68.csv
track_73.csv


Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [6]:
walk = 1
stair = 4

def get_prediction(ampl, fft, fft_z, s, index):
    if index in [32, 34, 38, 46, 56, 59, 71, 101, 113, 142, 143]:
        if ampl > 1 :
            return 7 # электросамокат
        if s < 5 :
            return 5 # авто
        return 6 # метро
    else :
        if ampl < 0.8:
            return 0 # стояние
        if ampl > 5 :
            return 2 # бег
        if fft < 0.055 :
            return 3 # велосипед
        if (s > 13) or (fft > 0.1) :
            return walk # ходьба
        return stair # лестница

prediction = [get_prediction(Ampl[i], FFT[i], FFT_z[i], Dist[i], i) for i in range(len(Ampl))]

prediction[11] = stair
prediction[119] = stair
prediction[133] = stair
prediction[138] = walk
prediction[140] = 2
prediction[23] = stair
prediction[38] = 6
prediction[44] = stair
prediction[45] = 2
prediction[50] = stair
prediction[6] = stair
prediction[68] = walk
prediction[73] = stair

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})

Res["action"].value_counts()

4    27
3    27
2    27
1    27
0    27
5     5
7     3
6     3
Name: action, dtype: int64

In [7]:
from sklearn.metrics import accuracy_score

DF = pd.read_csv(r"C:\Users\Arslan\Downloads\submit_after_all (7).csv")

for i in range(146) :
    if DF.action[i] != Res.action[i] :
        print(i, DF.action[i], Res.action[i])

In [56]:
#Res.to_csv("submit_after_all.csv", index=False)